In [1]:
import pandas as pd

# Wczytanie danych
file_path = "filtered_mbsa_full_clearedBERT_onlycleanedtext.csv"  # Zmień nazwę pliku na odpowiednią
data = pd.read_csv(file_path)

In [2]:
# Zamiana NaN na pusty ciąg znaków
data['text'] = data['text'].fillna('')

# Upewnienie się, że wszystkie wartości w kolumnie 'text' są typu string
data['text'] = data['text'].astype(str)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import pickle

# Mapowanie etykiet na liczby
label_mapping = {"Negative": 0, "Positive": 1, "Neutral": 2}
data['label'] = data['Sentiment'].map(label_mapping)

# Podział danych na zbiór treningowy i testowy
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['text'].tolist(),
    data['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=data['label']
)

# Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Funkcja tokenizująca z paskiem postępu
def tokenize_with_progress(texts, tokenizer, max_length=256):
    tokenized_data = []
    for text in tqdm(texts, desc="Tokenizacja", unit="text"):
        tokenized = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        tokenized_data.append({key: val.squeeze(0).tolist() for key, val in tokenized.items()})
    return tokenized_data

# Tokenizacja danych
print("Tokenizowanie zbioru treningowego...")
train_encodings = tokenize_with_progress(train_texts, tokenizer)

print("Tokenizowanie zbioru walidacyjnego...")
val_encodings = tokenize_with_progress(val_texts, tokenizer)

# Zapisanie ztokenizowanych danych do plików
with open("train_tokenized.pkl", "wb") as f:
    pickle.dump(train_encodings, f)

with open("val_tokenized.pkl", "wb") as f:
    pickle.dump(val_encodings, f)

print("Ztokenizowane dane zapisano do plików: train_tokenized.pkl i val_tokenized.pkl")

Tokenizowanie zbioru treningowego...


Tokenizacja:  93%|█████████▎| 8972085/9608460 [1:31:00<07:32, 1406.50text/s]  